<a href="https://colab.research.google.com/github/ShogoNoguchi/task_CSL/blob/main/make_7_1_20180411_cnt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

7_1_20180411.cntだけ処理ができませんでしたので、これのみ試行します。

In [1]:
# DagsHubとboto3のインストール
%pip install -q dagshub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.2/252.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.6 MB/s eta 0:00:00


In [3]:
import dagshub.colab
DAGSHUB_REPO = dagshub.colab.login()

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=68a58f79-188e-4394-ae26-ee88952e7a55&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=948cc21f9eeabd9eea4e051a540a318726854bb30ae2ea633b366dc4da247e18&referrer=colab




Accessing as t220g139

Repository t220g139/dagshub-drive is ready for use with Colab. Link to the repository:
https://dagshub.com/t220g139/dagshub-drive


In [4]:
mount_path = dagshub.storage.mount(DAGSHUB_REPO)

Do you want to install all missing packages?
Are you sure [y/(N)]: y


Installing rclone...

rclone installed successfully.

Installing fusermount3...

fusermount3 installed successfully.

Successfully mounted DagsHub Storage in 'dagshub-drive' to 'dagshub-drive/dagshub_storage'.

To unmount, run `dagshub.storage.unmount(repo="t220g139/dagshub-drive", path="dagshub-drive/dagshub_storage")`.

In [7]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.7 MB/s eta 0:00:00


In [11]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
# MNEのインストール（既にインストール済みの場合はスキップ）
!pip install mne

# 必要なライブラリのインポート
import mne
import numpy as np
import os
import pickle
import logging
from datetime import datetime, timezone
import dagshub.colab
from glob import glob


In [13]:
# ディレクトリの設定
output_dir = "/content/processed/"
os.makedirs(output_dir, exist_ok=True)
for folder in ["processed_train", "processed_eval", "processed_test"]:
    folder_path = os.path.join(output_dir, folder)
    os.makedirs(folder_path, exist_ok=True)

# ログ設定
logger = logging.getLogger('EEG_Processing_Specific_File')
logger.setLevel(logging.INFO)

# ログフォーマットの設定
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# ファイルハンドラの作成
log_file_path = os.path.join(output_dir, 'processing_specific_file.log')
file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

# ストリームハンドラの作成（コンソール出力）
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(formatter)

# ハンドラをロガーに追加（重複追加を防ぐ）
if not logger.handlers:
    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)


In [16]:
# 処理対象のファイル名とパス
file_to_process = "7_1_20180411.cnt"
file_path = "/content/drive/MyDrive/SEEDV/EEG_raw/7_1_20180411.cnt"

# サンプリング周波数
sfreq = 200


In [17]:
# 不要なチャンネルのリスト
useless_ch = ['M1', 'M2', 'VEO', 'HEO', 'EKG', 'ECG']

# 標準チャンネル順序
chOrder_standard = [
    'FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1',
    'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6',
    'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ',
    'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2'
]

# トライアル境界（秒単位）
session_starts = {
    1: [30, 132, 287, 555, 773, 982, 1271, 1628, 1730, 2025, 2227, 2435, 2667, 2932, 3204],
    2: [30, 299, 548, 646, 836, 1000, 1091, 1392, 1657, 1809, 1966, 2186, 2333, 2490, 2741],
    3: [30, 353, 478, 674, 825, 908, 1200, 1346, 1451, 1711, 2055, 2307, 2457, 2726, 2888]
}
session_ends = {
    1: [102, 228, 524, 742, 920, 1240, 1568, 1697, 1994, 2166, 2401, 2607, 2901, 3172, 3359],
    2: [267, 488, 614, 773, 967, 1059, 1331, 1622, 1777, 1908, 2153, 2302, 2428, 2709, 2817],
    3: [321, 418, 643, 764, 877, 1147, 1284, 1418, 1679, 1996, 2275, 2425, 2664, 2857, 3066]
}

# セッションごとの感情ラベル
emotions_session1 = ["happy", "fear", "neutral", "sad", "disgust", "happy", "fear", "neutral", "sad", "disgust", "happy", "fear", "neutral", "sad", "disgust"]
emotions_session2 = ["sad", "fear", "neutral", "disgust", "happy", "happy", "disgust", "neutral", "sad", "fear", "neutral", "happy", "fear", "sad", "disgust"]
emotions_session3 = ["sad", "fear", "neutral", "disgust", "happy", "happy", "disgust", "neutral", "sad", "fear", "neutral", "happy", "fear", "sad", "disgust"]

# 感情ラベルのマッピング
emo2id = {"disgust": 0, "fear": 1, "sad": 2, "neutral": 3, "happy": 4}

# スキップされたファイルを記録するリスト
skipped_files = []
processed_files_count = 0

# 新しく生成された.pklファイルを記録するリスト
new_pkl_files = []

# スクリプトの開始をログに記録
logger.info(f"EEGデータ処理を開始します。対象ファイル: {file_to_process}")

# ファイルが存在するか確認
if not os.path.exists(file_path):
    logger.error(f"ファイルが存在しません: {file_path}. スキップします。")
    skipped_files.append(file_to_process)
else:
    fname = os.path.basename(file_path)
    parts = fname.split('_')

    # ファイル名から被験者IDとセッションIDを解析
    if len(parts) < 3:
        logger.warning(f"Filename {fname} does not conform to expected format. Skipping this file.")
        skipped_files.append(fname)
    else:
        subject_id = parts[0]
        try:
            ses_id = int(parts[1])
        except ValueError:
            logger.warning(f"Session ID in filename {fname} is not an integer. Skipping this file.")
            skipped_files.append(fname)
            ses_id = None

        # 日付部分を抽出（例：7_1_20180411.cnt -> 20180411）
        date_str = parts[-1].split('.')[0]
        try:
            meas_datetime = datetime.strptime(date_str, '%Y%m%d')
        except ValueError as e:
            logger.warning(f"Date parsing failed for {fname}: {e}. Setting meas_date to None.")
            meas_datetime = None

        if ses_id is not None:
            # セッションIDに対応する開始・終了時間を取得
            starts = session_starts.get(ses_id)
            ends = session_ends.get(ses_id)
            if not starts or not ends:
                logger.warning(f"No session boundaries defined for session {ses_id} in {fname}. Skipping.")
                skipped_files.append(fname)
            else:
                # セッションに対応する感情ラベルの取得
                emotions = {
                    1: emotions_session1,
                    2: emotions_session2,
                    3: emotions_session3
                }.get(ses_id, [])

                if len(emotions) != 15:
                    logger.warning(f"Emotion list length mismatch for session {ses_id} in {fname}. Skipping.")
                    skipped_files.append(fname)
                else:
                    # .cntファイルの読み込み（date_formatとdata_formatを適切に処理）
                    read_success = False
                    for fmt in ['mm/dd/yy', 'dd/mm/yy']:  # 'auto'を除外
                        for data_fmt in ['int16', 'int32']:  # 'auto'と'float32'を除外
                            try:
                                raw = mne.io.read_raw_cnt(
                                    file_path,
                                    preload=True,
                                    date_format=fmt,
                                    data_format=data_fmt,
                                    verbose='WARNING'  # 警告を表示
                                )
                                logger.info(f"Successfully loaded {fname} with date_format='{fmt}' and data_format='{data_fmt}'.")
                                read_success = True
                                break
                            except ValueError as e:
                                logger.warning(f"Failed to load {fname} with date_format='{fmt}' and data_format='{data_fmt}': {e}. Trying next format.")
                            except Exception as e:
                                logger.error(f"Unexpected error while loading {fname} with date_format='{fmt}' and data_format='{data_fmt}': {e}.")
                                break  # その他のエラーの場合はループを抜ける
                        if read_success:
                            break

                    if not read_success:
                        logger.error(f"Failed to load {fname} with any supported date_format and data_format. Skipping this file.")
                        skipped_files.append(fname)
                    else:
                        # 測定日が設定されていない場合、手動で設定
                        if raw.info['meas_date'] is None and meas_datetime is not None:
                            try:
                                # meas_datetime を UTC に変換
                                meas_datetime_utc = meas_datetime.replace(tzinfo=timezone.utc)
                                # Unixタイムスタンプ（秒）を取得
                                meas_timestamp = meas_datetime_utc.timestamp()
                                raw.set_meas_date(meas_timestamp)
                                logger.info(f"Set meas_date for {fname} to {meas_datetime_utc} (UTC).")
                            except Exception as e:
                                logger.warning(f"Failed to set meas_date for {fname}: {e}. Proceeding without it.")
                        elif raw.info['meas_date'] is None and meas_datetime is None:
                            logger.info(f"meas_date is not set for {fname}, proceeding without it.")

                        # 不要チャンネル除去
                        available_ch = raw.ch_names
                        drop_ch = [ch for ch in useless_ch if ch in available_ch]
                        if drop_ch:
                            raw.drop_channels(drop_ch)
                            logger.info(f"Dropped channels {drop_ch} from {fname}.")

                        # 必要チャンネル確認
                        missing_ch = [ch for ch in chOrder_standard if ch not in raw.ch_names]
                        if missing_ch:
                            logger.warning(f"Missing channels {missing_ch} in {fname}. Skipping this file.")
                            skipped_files.append(fname)
                        else:
                            # チャンネル並び替え
                            try:
                                raw.reorder_channels(chOrder_standard)
                                logger.info(f"Reordered channels for {fname}.")
                            except Exception as e:
                                logger.error(f"Failed to reorder channels for {fname}: {e}. Skipping this file.")
                                skipped_files.append(fname)

                            # フィルタリング
                            try:
                                logger.info(f"Filtering raw data for {fname}: Band-pass 0.1 - 75 Hz, Notch 50 Hz")
                                raw.filter(l_freq=0.1, h_freq=75.0, fir_design='firwin')
                                raw.notch_filter(freqs=50.0, fir_design='firwin')
                                raw.resample(sfreq, n_jobs=1)
                                data = raw.get_data(units='uV')
                                logger.info(f"Filtered and resampled data for {fname}.")
                            except Exception as e:
                                logger.error(f"Filtering or resampling failed for {fname}: {e}. Skipping this file.")
                                skipped_files.append(fname)

                            # 各トライアルの処理
                            try:
                                for t_i in range(15):
                                    start_sample = starts[t_i] * sfreq
                                    end_sample = ends[t_i] * sfreq
                                    trial_data = data[:, start_sample:end_sample]
                                    y = emo2id.get(emotions[t_i], -1)
                                    if y == -1:
                                        logger.warning(f"Unknown emotion '{emotions[t_i]}' in {fname}. Skipping trial.")
                                        continue

                                    length = trial_data.shape[1]
                                    num_segments = length // 200

                                    trial_id = t_i + 1
                                    if trial_id <= 5:
                                        split_folder = "processed_train"
                                    elif trial_id <= 10:
                                        split_folder = "processed_eval"
                                    else:
                                        split_folder = "processed_test"

                                    for seg_i in range(num_segments):
                                        seg_data = trial_data[:, seg_i * 200:(seg_i + 1) * 200]
                                        sample_dict = {"X": seg_data, "y": y, "ch_names": chOrder_standard}
                                        out_name = f"{subject_id}_{ses_id}_{trial_id}_{seg_i}.pkl"
                                        out_path = os.path.join(output_dir, split_folder, out_name)
                                        with open(out_path, "wb") as f:
                                            pickle.dump(sample_dict, f)
                                        logger.info(f"Processed trial {trial_id}, segment {seg_i} of {fname}.")
                                        new_pkl_files.append(out_path)
                                processed_files_count += 1
                                logger.info(f"Successfully processed {fname}.")
                            except IndexError as e:
                                logger.error(f"Index error in trial {t_i+1} for {fname}: {e}. Skipping this trial.")
                            except Exception as e:
                                logger.error(f"Unexpected error in trial {t_i+1} for {fname}: {e}. Skipping this trial.")


2024-12-09 08:09:27,403 - INFO - EEGデータ処理を開始します。対象ファイル: 7_1_20180411.cnt
INFO:EEG_Processing_Specific_File:EEGデータ処理を開始します。対象ファイル: 7_1_20180411.cnt
2024-12-09 08:10:50,561 - WARNING - Failed to load 7_1_20180411.cnt with date_format='mm/dd/yy' and data_format='int16': cannot reshape array of size 15773870 into shape (757575,66,1). Trying next format.
2024-12-09 08:11:50,020 - WARNING - Failed to load 7_1_20180411.cnt with date_format='mm/dd/yy' and data_format='int32': cannot reshape array of size 7887232 into shape (378787,66,1). Trying next format.
2024-12-09 08:12:56,091 - WARNING - Failed to load 7_1_20180411.cnt with date_format='dd/mm/yy' and data_format='int16': cannot reshape array of size 15773870 into shape (757575,66,1). Trying next format.
2024-12-09 08:13:53,719 - WARNING - Failed to load 7_1_20180411.cnt with date_format='dd/mm/yy' and data_format='int32': cannot reshape array of size 7887232 into shape (378787,66,1). Trying next format.
2024-12-09 08:13:53,866 - ERROR - F